In [1]:
#move to the folder where the scraped data is on your device
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Data/WiChat

Mounted at /content/drive
/content/drive/MyDrive/Data/WiChat


In [2]:
!pip install virtualenv
!virtualenv /content/drive/MyDrive/Data/WiChat
!source /content/drive/MyDrive/Data/WiChat/bin/activate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 27.1 MB/s eta 0:00:00
created virtual environment CPython3.11.11.final.0-64 in 32753ms
  creator CPython3Posix(dest=/content/drive/MyDrive/Data/WiChat, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.0.1, setuptools==75.8.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
# install dependencies
!pip install -qU langchain
!pip install -qU transformers
!pip install -qU langchain-community
!pip install -qU unstructured
!pip install -qU sentence-transformers
!pip install -qU faiss-cpu
!pip install -qU langchain_openai
!pip install -qU openai

In [10]:
import openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser


In [16]:
#API_KEY = os.environ["OPENAI_API_KEY"]
#openai.apikey = APIKEY
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = key
# ensure the notebook is in the same folder as the data files
# load all txt files
def load_data(path):
  loader1 = DirectoryLoader(path, glob = '*.txt', show_progress = True)
  # get content of txt files
  docs = loader1.load()

  return docs


def get_chunks(docs):

    # split the txt files into chunks of 1000 characters and 150 characters overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 150)
    chunks = text_splitter.split_documents(docs)


    return chunks

path = '/content/drive/MyDrive/Data/WiChat/scraped_data'
docs = load_data(path)

data = get_chunks(docs)

100%|██████████| 79/79 [00:04<00:00, 18.89it/s]


In [8]:


# embed data sources
def embed(data, device, model):
  model_kwargs = {'device': device}
  encode_kwargs = {'normalize_embeddings': False}

  embeddings = HuggingFaceEmbeddings(
    model_name = model,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
  )
  return embeddings

def store_data(data, embeddings):
  # vector store
  db = FAISS.from_documents(data, embeddings)
  return db

embeddings = embed(data, 'cpu', 'sentence-transformers/all-MiniLM-L6-v2')
db = store_data(data, embeddings)



<ipython-input-8-ca2f2b160cc8>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
llm = ChatOpenAI(model = "gpt-4o")

# Define the chat prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are called WiChat, which is short for Worldbank Ideas Chatbot, the chatbot for the Worldbank Ideas Project. You are friendly and follow instructions to answer questions extremely well. Please be truthful and give direct answers. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the response short and concise in at most five sentences. If the user chats in a different language, translate accurately and respond in the same language. You will provide specific details and accurate answers to user queries on the Worldbank Ideas Project."),
         MessagesPlaceholder("chat_history"),
        ("human", "Use the user question {input} to answer the question. Use only the retrieved{context} to answer the question.")
    ]
)

# --- Create RAG chain ---

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
### Statefully manage chat history ###

messages_history = {}

def get_session_history(session_id: str):
    if session_id not in messages_history:
        messages_history[session_id] = ChatMessageHistory()
    return messages_history[session_id]



In [27]:
retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


# --- Response Generation ---
def generate_response(query):

    return conversational_rag_chain.invoke({"input": query}, config={"configurable": {"session_id": "1"}})["answer"]

query = "What is worldbank ideas project"
response = generate_response(query)


response

'The World Bank IDEAS Project aims to address deficiencies in the Nigerian education system that contribute to unemployment among school leavers. The project encourages Nigerian youths to seize the opportunities it offers. It involves an estimated 40 technical colleges and includes a technical teacher training component, working alongside the private sector. A $200 million credit facility from the World Bank has been approved for this project, which will be implemented over five years.'

In [28]:

query = "Why are you called WiChat?"
response = generate_response(query)


response

"I'm called WiChat because I'm a friendly chatbot created for the Worldbank Ideas Project to provide information and support users interested in the project's initiatives."

In [29]:
query = "World Bank Ideas project, c'est quoi?"
response = generate_response(query)


response

"Le projet IDEAS de la Banque mondiale vise à remédier aux insuffisances du système éducatif en Nigeria, ayant conduit à un grand nombre de jeunes diplômés au chômage. Le projet bénéficie à environ 40 collèges techniques à travers le pays ainsi qu'au secteur privé, et comprend aussi une composante de formation pour les enseignants techniques. Un financement de deux cents millions de dollars de la Banque mondiale a été approuvé pour son déroulement, prévu sur une période de cinq ans. Les jeunes Nigérians sont encouragés à profiter pleinement des opportunités offertes par ce projet."